In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r"data\export_with_days_delta.xlsx")

In [3]:
df

,clean_id,raw_id,clinic_application_date,intake_date,signing_date,suiteable_for_pp,First name,sheet,group,therapy_start_date,...,Randomized,Dropout,Research Dropout,Clinical Dropout,In Waiting List,Finished,Active,Not Cooperative,waiting_duration,did_started_therapy
0,329822563,329822563,2025-03-16 00:00:00,2025-03-16 00:00:00,0,0,בר,נשירה מחקרית,CAU,2025-11-04 00:00:00,...,True,True,True,False,False,False,False,False,233.0,True
1,a0017,a0017s,2025-03-12 00:00:00,2025-04-06 00:00:00,0,1,עמית,פספוסי גיוסים,0,0,...,False,False,False,False,False,False,False,False,NaN,False
2,a0226,a0226s,2025-01-07 00:00:00,2025-01-14 00:00:00,0,1,אדירם,אי התאמה למחקר,0,0,...,False,False,False,False,False,False,False,False,NaN,False
3,a0531,a0531s,2025-01-21 00:00:00,2025-02-06 00:00:00,0,1,אבישי,אי הסכמה למחקר,CAU,0,...,False,False,False,False,False,False,False,False,NaN,False
4,a0556,a0556s,2025-04-20 00:00:00,2025-05-11 00:00:00,0,1,עתי,פספוסי גיוסים,0,0,...,False,False,False,False,False,False,False,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,y9000,y9000s,2024-09-23 00:00:00,2025-01-14 00:00:00,0,1,יובל,אי התאמה למחקר,0,2025-04-22 00:00:00,...,False,False,False,False,False,False,False,False,98.0,True
302,y9212,Y9212,2025-03-25 00:00:00,2025-03-25 00:00:00,2025-03-25 00:00:00,1,ירדן,משתתפים פעילים,CAU,0,...,True,False,False,False,True,False,True,False,NaN,False
303,y9411,y9411s,2024-12-16 00:00:00,2025-03-20 00:00:00,0,0,ירון,נשירה קלינית- לאחר ת. טיפול,Stepped Care,2025-05-08 00:00:00,...,True,True,False,True,False,False,False,False,49.0,True
304,z1585,Z1585s,2025-02-27 00:00:00,2025-04-20 00:00:00,2025-04-20 00:00:00,לא- הומלץ רק שאלונים,זיו,משתתפים פעילים,Stepped Care,0,...,True,False,False,False,True,False,True,False,NaN,False


In [4]:
consort_groups = [
    'N',
    'Eligible',
    'Randomized',
    'Dropout',
    'Research Dropout',
    'Clinical Dropout',
    'In Waiting List',
    'Finished',
    'Active',
    'Not Cooperative'
    ]

In [5]:
df.clean_id.nunique() == df.shape[0]

True

In [6]:
df[(df['Randomized'] == True) & (df['group'] == 'CAU')].shape

(58, 34)

In [7]:
results = []

for i in consort_groups:
    row = {
        'consort_group': i,
        'size': df[i].sum(),
        'participants': df[df[i]].clean_id,   
    }
    results.append(row)

In [8]:
df.group.unique()

array(['CAU', '0', 'Stepped Care', 'טיפול בקבוצת הורים '], dtype=object)

In [9]:
group_results = []

for group in ['CAU', 'Stepped Care', None]:
    if group is None:
        grouped_df = df[~df.group.isin(['CAU', 'Stepped Care'])]
    else:
        grouped_df = df[df.group == group]
        
    print(group, grouped_df.shape)
    for i in consort_groups:
        row = {
            'consort_group': i,
            'size': grouped_df[i].sum(),
            'participants': grouped_df[grouped_df[i]].clean_id,  
            'experiment_group': group
        }
        group_results.append(row)

CAU (64, 34)
Stepped Care (61, 34)
None (181, 34)


In [10]:
cau = [i for i in group_results if i['experiment_group'] == 'CAU']
ipc = [i for i in group_results if i['experiment_group'] == 'Stepped Care']
non = [i for i in group_results if i['experiment_group'] is None]

In [11]:
pd.DataFrame(results).to_csv("consort_group_sizes.csv", index=0)

In [12]:
pd.DataFrame(cau).to_csv("consort_groups_sizes_CAU.csv", index=0)

In [13]:
pd.DataFrame(ipc).to_csv("consort_group_sizes_Stepped_Care.csv", index=0)

In [14]:
pd.DataFrame(non).to_csv("consort_sizes_no_group.csv", index=0)

In [15]:
start = "first_contact_date"
end = 'therapy_starting_date'

In [29]:
(df[start] == '0').value_counts()
    
    
(df[end] == '0').value_counts()

therapy_starting_date
False    306
Name: count, dtype: int64

In [30]:
ended_df = df[(df[end] != '0') & (df[start] != '0')][['clean_id', 'group', start, end]]

In [31]:
ended_df['waiting_duration'] =  pd.to_datetime(ended_df[end]) - pd.to_datetime(ended_df[start])

In [32]:
ended_df.to_csv("patients_with_started_therapy.csv")

In [33]:
ended_df.to_csv("waiting_time_duration.csv")

In [34]:
'CAU', 'Stepped Care'

('CAU', 'Stepped Care')

In [35]:
ended_df_cau = ended_df[ended_df.group == 'CAU']

In [36]:
ended_df_ipc = ended_df[ended_df.group == 'Stepped Care']

In [37]:
ended_df_no_group =  ended_df[~ended_df.group.isin(['CAU', 'Stepped Care'])]

In [38]:
pd.to_datetime(ended_df_cau[end]) - pd.to_datetime(ended_df_cau[start])

0     233 days
3          NaT
5     156 days
7          NaT
11    175 days
        ...   
278        NaT
281     0 days
282    47 days
286        NaT
302        NaT
Length: 64, dtype: timedelta64[ns]

In [39]:
pd.to_datetime(ended_df_ipc[end]) - pd.to_datetime(ended_df_ipc[start])

6          NaT
8      21 days
12     53 days
13         NaT
14     11 days
        ...   
289   104 days
295   166 days
300        NaT
303    49 days
304        NaT
Length: 61, dtype: timedelta64[ns]

In [40]:
pd.to_datetime(ended_df_no_group[end]) - pd.to_datetime(ended_df_no_group[start])

1         NaT
2         NaT
4         NaT
9         NaT
10        NaT
        ...  
297       NaT
298       NaT
299       NaT
301   98 days
305       NaT
Length: 181, dtype: timedelta64[ns]

In [41]:
df.columns

Index(['clean_id', 'raw_id', 'clinic_application_date', 'intake_date',
       'signing_date', 'suiteable_for_pp', 'First name', 'sheet', 'group',
       'therapy_start_date', 'therapy_end_date', 'אי הסכמה למחקר',
       'אי התאמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית',
       'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2',
       'פספוסי גיוסים', 'first_contact_date', 'therapy_starting_date', 'N',
       'Eligible', 'Randomized', 'Dropout', 'Research Dropout',
       'Clinical Dropout', 'In Waiting List', 'Finished', 'Active',
       'Not Cooperative', 'waiting_duration', 'did_started_therapy'],
      dtype='object')